## Homework 6

We want to solve the linear system given by 
\begin{equation}
Ax = b,
\end{equation}
where $A$ is a non-singular $n\times n$ matrix. 

You have already solved this problem using Gaussian Elimination (and its partial pivoted version) which has an assymptotic cost $\mathcal{O}(n^3)$. 

In this homework you will try to solve the system in lower complexity using an iterative method. In particular, you will implement the Jacobi and Gauss-Seidel iterations and you will study its limitations.

Both Jacobi and Gauss-Iterations can be seen as fixed point methods, used by a matrix splitting. 

#### Matrix Splitting

Given a square matrix $A$ you will define a splitting in two matrices $N$ and $M$ such that $A = N+M$.
If you suppose that $N$ is invertible you can write the system 
\begin{equation}
Ax = b,
\end{equation}
as 
\begin{equation}
Nx =  b - Mx,
\end{equation}
and you can define the fixed point iteration as
\begin{equation}
Nx^{n+1} =  b - Mx^{n},
\end{equation}
or equivalently
\begin{equation}
x^{n+1} = N^{-1}\left ( b - Mx^{n} \right).
\end{equation}

In this case you can show that the convergence speed is can be determined from the spectral radius of the iteration matrix:
\begin{equation}
T =  N^{-1}M.
\end{equation}

### Question 1: Jacobi Iteration

The Jacobi iteration correspond to a fixed point method, in which the Matrix splitting is given by 
\begin{equation}
N = D, \qquad M= L+U,
\end{equation}
where $D$ is the diagonal of $A$ and $L$ and $U$ are the (strictly) upper and lower triangular parts of $A$.

In order to extract the diagonal, upper triangular and lower triangular matrices from $A$ we will use the built-in functions in Julia. 


In [3]:
# size of the matrices
m = 10
# generate a random matrix
A = rand(m,m) + m*eye(m)
# get the digonal part
D = diagm(diag(A),0)
# to get the upper triangular part of the matrix
U = triu(A,1)
#and the lower part 
L = tril(A,-1)
# we check that this is indeed a matrix splitting
println("Error in the splitting = ",norm(A -(L+D+U)))

Error in the splitting = 0.0


Q1.a Implement the Jacobi iteration with input:
- $A$ a square matrix,
- $b$ a vector <br>

Your function will have the following optional parameters
- $Nmax$ maximum number of iterations, by default set to 30
- $\epsilon$ the tolerance, by default set to 1e-6
- history this is a boolean to return all the succesive approximations <br> 

The ouput of your function is the final approximation $x$ of your vector, of in the case that history is true, it will output a matrix with the all the intermediate approximations of size $ n \times \mbox{number of iterations to converge}$.

Your function will raise an error if it didn't converge in the $Nmax$ iterations.

Hint:
- to build the matrix with the intermediat steps you can use hcat

In [5]:
function JacobiIt(A,b; Nmax = 30, ϵ=1e-6, history = false)
    (n,m) = size(A)
    n!=m && error("Dimension mistmach")
    # initial guess
    x0 = zeros(b)
    history && (xHist = x0)
    absB = norm(b)
    # D
    D = diagm(diag(A),0)
    # L+U
    M = triu(A,1) + tril(A,-1)
    for i = 1:Nmax
        x = D\(b - M*x0)
        history && (xHist = hcat(xHist,x))
        if norm(x-x0)/absB < ϵ
            history ? (return xHist[:,2:end]) : (return x)
        end
        x0 = x
    end
    error("Tolerance not achieved within the maximum number of iterations")
end

JacobiIt (generic function with 1 method)

In [6]:
A = rand(m,m) + m*eye(m)
b = rand(m,1)
X = JacobiIt(A,b, history = true)
size(X,2)

15

### Question 2: Gauss-Seidel iteration

The Gauss-Seidel iteration is analogous to the Jacobi iteration; however, it uses a different splitting, 
namely 
\begin{equation}
N = D+U, \qquad M= L,
\end{equation}

Q1.a Implement the Gauss-Seidel iteration with input:
- $A$ a square matrix,
- $b$ a vector <br>

Your function will have the following optional parameters
- $Nmax$ maximum number of iterations, by default set to 30
- $\epsilon$ the tolerance, by default set to 1e-6
- history this is a boolean to return all the succesive approximations <br> 

The ouput of your function is the final approximation $x$ of your vector, of in the case that history is true, it will output a matrix with the all the intermediate approximations of size $ n \times \mbox{number of iterations to converge}$

Your function will raise an error if it didn't converge in the $Nmax$ iterations.

Hint:
- to build the matrix with the intermediat steps you can use hcat

In [12]:
function GaussSeidelIt(A,b; Nmax = 30, ϵ=1e-6, history = false)
    x0 = zeros(b)
    history && (xHist = x0)
    absB = norm(b)
    N = triu(A,0)
    # to get the upper triangular part of the matrix
    M = tril(A,-1)
    for i = 1:Nmax
        x = N\(b - M*x0)
        history && (xHist = hcat(xHist,x))
        if norm(x-x0)/absB < ϵ
            history ? (return xHist[:,2:end]) : (return x)
        end
        x0 = x
    end
    error("Tolerance not achieved within the maximum number of iterations")
end

GaussSeidelIt (generic function with 1 method)

In [13]:
A = rand(m,m) + m*eye(m)
b = rand(m,1)
X = GaussSeidelIt(A,b, history = true)
size(X,2)

6

### Question 3: Comparison 

Use your algorithms to solve the following randomly generated system


In [14]:
m = 100
A = rand(m,m) + m*eye(m)
b = rand(m,1)

@time XJac = JacobiIt(A,b, history = true)
println("The residual is : ", norm(A*XJac[:,end]- b)/norm(b) )
println("number of iterations is : ",size(XJac,2))

@time XGS = GaussSeidelIt(A,b, history = true)
println("The residual is : ", norm(A*XGS[:,end]- b)/norm(b) )
println("number of iterations is : ",size(XGS,2))


  0.002490 seconds (347 allocations: 506.011 KB)
The residual is : 4.240519883754411e-5
number of iterations is : 14
  0.000487 seconds (99 allocations: 675.609 KB)
The residual is : 1.5830214054637808e-6
number of iterations is : 6


Q3.a How can you explain the one converges faster than the other? Write a small script that computes the spectral radius of the iteration matrix for both algorithms and use it on your explanation. 

In [ ]:
# write your script in here

Answer

Now you will use your algorithms to solve a slighly different system

In [121]:
m = 100;
A = rand(m,m) + sqrt(m)*eye(m);
b = rand(m,1);

With the Jacobi iteration, 

In [122]:
@time XJac = JacobiIt(A,b, history = true)
println("number of iterations is : ",size(XJac,2))

LoadError: LoadError: Tolerance not achieved within the maximum number of iterations
while loading In[122], in expression starting on line 155

and with the Gauss Seidel iteration

In [123]:
@time XGS = GaussSeidelIt(A,b, history = true)
println("number of iterations is : ",size(XGS,2))

  0.000570 seconds (218 allocations: 1.446 MB)
number of iterations is : 15


Q3.b How can you explain the one converges and the other just fails? Write a small script that computes the spectral radius of the iteration matrix for both algorithms and use it on your explanation. 

In [113]:
# write your script in here

10.001731157500526

Answer:

### Question 4: Complexity (bonus) (10 points)


Q4.a what is the complexity of one iteration of the Gauss-Seidel method? and about Jacobi?

Answer:

Q4.b What would be the condition on the number of iterations in order such that the Jacobi and Gauss-Seidel iteration would have a better complexity that Gaussian Elimination

Answer:

Q4.c Run a benchmark with the randomly generated systems above. Can you achieve quadratic convergence? 

In [136]:
# write your whole script here